# We are going to make a sales brochure generator wioth gradio UI - and with this we wont have to change the url everywhere. 

So we are gonna start with basics. We have scrapper.py. In that we have fetch links, and fetch content - two methods. 
1. We get the links (all links) from the url we get.
2. We give an llm the list of links to get the important links only.
3. We create a string - with links and their page content( we can use fetch_content) - and we do this for main url. And then the relevant links urls. we add their contents too.
4. Now we use our main llm and get the sales brochure.
5. Add gradio in this overall 

In [11]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import sys

In [27]:
import gradio as gr

In [8]:
sys.path.append(os.path.abspath(".."))
from Week1.scrapper import fetch_website_links, fetch_website_contents
from IPython.display import Markdown, display, update_display

In [9]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [12]:
openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

In [23]:
system_prompt_links = """
You are an helping assisstant who when recieves a list of links from a company/person's website - you will consider all the links. 
And you will return only the links that are improtant and must be added to the sales brochure created for prospective customers, investors and recruits.
These important links should cover main pages that must be added in sales brochure like about page, contact us, or blog - or main content. 

Reply in JSON format.
Example of a reply - 
Links = [
    {"Type":"About", "Link":"<the link>"},
    {"Type":"Careers", "Link":"<the link>"},
]
"""

def get_user_prompt_for_links(url):
    user_prompt = """
    Here is the list of links from the website. Please go through these and select the main links that can be added to the sales brochure. 
    """
    user_prompt +=  "\n".join(fetch_website_links(url))
    return user_prompt

def get_imp_links_llm(url):
    message = [
        {"role":"system", "content":system_prompt_links},
        {"role":"user", "content":get_user_prompt_for_links(url)}
    ]
    response = openai.chat.completions.create( model="gpt-5-nano", messages=message, response_format={"type": "json_object"})
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links)} relevant links")
    return links

def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = get_imp_links_llm(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['Links']:
        result += f"\n\n### Link: {link['Type']}\n"
        result += fetch_website_contents(link["Link"])
    return result

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [24]:
def create_brochure_gpt(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    return result

In [18]:
def create_brochure_claude(company_name, url):
    response = anthropic.chat.completions.create(
        model="claude-sonnet-4-5-20250929",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    return result

In [21]:
def select_model(name, url, model):
    if model=="GPT":
        result = create_brochure_gpt(name, url)
    elif model=="Claude":
        result = create_brochure_claude(name, url)
    else:
        raise ValueError("Unknown model")
    return result

In [28]:
company_name = gr.Textbox(label="Your Company Name:", info="Enter a company name", lines=1)
company_url = gr.Textbox(label="Your Company URL:", info="Enter a company url", lines=2)
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=select_model,
    title="LLMs", 
    inputs=[company_name, company_url, model_selector],
    outputs=[message_output], 
    examples=[
            ["Indeed | https://in.indeed.com/?r=us", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


Found 1 relevant links
Found 1 relevant links
